# Amazon SageMaker - Bring Your Own Model 
## TensorFlow + Keras 編

ここでは TensorFlow と Keras を使ったサンプルコードを題材に、Amazon SageMaker 移行の方法を順を追って説明します。SageMaker Python SDK で TensorFlow を使うための説明は [SDK のドキュメント](https://sagemaker.readthedocs.io/en/stable/using_tf.html) にも多くの情報があります。

注: 
ここで説明するのは Script モード という記法 (現時点では標準の書き方) で、FILE モード (入力データを Amazon S3 から学習時にファイルとしてコピーする方法) です。データサイズが大きくなった場合は、FILE Mode ではなく PIPE Mode をお使い頂いた方がスループットが向上します。
また、ここでは以降手順の紹介のためトレーニングスクリプトは最小限の書き換えとしています。

## 1. トレーニングスクリプトの書き換え
まず [サンプルのソースコード](https://github.com/keras-team/keras/blob/master/examples/mnist_mlp.py) を以下のコマンドでダウンロードします。

In [ ]:
!wget https://raw.githubusercontent.com/keras-team/keras/master/examples/mnist_mlp.py

このブログにあるように書き換えます。

https://aws.amazon.com/jp/blogs/news/amazon-sagemaker-keras/

## 2. Notebook 上でのデータ準備

トレーニングスクリプトの書き換えは終了しました。トレーニングを始める前に、予め Amazon S3 にデータを準備しておく必要があります。この Notebook を使ってその作業をします。

In [ ]:
import os
import keras
import numpy as np
from keras.datasets import mnist

import sagemaker

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

os.makedirs("./data", exist_ok = True)

np.savez('./data/train', image=x_train, label=y_train)
np.savez('./data/test', image=x_test, label=y_test)

In [ ]:
sagemaker_session = sagemaker.Session()
bucket_name = sagemaker_session.default_bucket()

input_data = sagemaker_session.upload_data(path='./data', bucket=bucket_name, key_prefix='data/handson-byom-tensorflow-keras')

print('Input data is uploaded to: {}'.format(input_data))

In [ ]:
from sagemaker.tensorflow import TensorFlow
from sagemaker import get_execution_role

role = get_execution_role()
estimator = TensorFlow(entry_point = "./mnist_mlp.py",
                       role=role,
                       train_instance_count=1,
                       train_instance_type="ml.m5.xlarge",
                       framework_version="1.12.0",
                       py_version='py3',
                       script_mode=True,
                       hyperparameters={'batch-size': 64,
                                        'num-classes': 10,
                                        'epochs': 4})

estimator.fit(input_data)